In [1]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse
import time

# 起点 URL
base_url = "https://www.musashino-u.ac.jp/"  # 武蔵野大学Webサイトのトップページ

# 辿ったURLとタイトルを保存するための辞書
url_title_dict = {}

# リンクを巡回するためのセット
to_visit = set([base_url]) 
visited = set() 

# ロボット除けとユーザーエージェントの設定
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Accept-Language': 'ja,en-US;q=0.9,en;q=0.8',
    'Accept-Encoding': 'gzip, deflate, br'
}

CRAWL_DELAY_SECONDS = 0.8

def normalize_url(url):
    """
    URLを正規化する関数。
    重複を避けるため、クエリパラメータとフラグメントを除去します。
    """
    parsed = urlparse(url)
    return urljoin(f"{parsed.scheme}://{parsed.netloc}", parsed.path)

def get_title(url):
    """ページの<title>を取得する関数"""
    try:
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status() # HTTPエラー（4xx, 5xx）があれば例外を発生させる
        time.sleep(CRAWL_DELAY_SECONDS) # 各リクエスト後に待機
        soup = BeautifulSoup(response.text, "html.parser")
        # タイトルがNoneでないことと、文字列であることを確認し、空白を除去
        title = soup.title.string.strip() if soup.title and soup.title.string else "No Title"
        # 連続する空白等を正規化 (オプション)
        return ' '.join(title.split())
    except requests.exceptions.RequestException as e:
        print(f"Error accessing {url}: {e}")
        return f"Fetching Error: {type(e).__name__}"
    except Exception as e:
        print(f"Error processing {url}: {e}")
        return f"Processing Error: {type(e).__name__}"

def find_links(url):
    """指定したURLに存在する、同一ドメイン内のリンクをすべて取得"""
    links = set()
    try:
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status() # HTTPエラー（4xx, 5xx）があれば例外を発生させる
        time.sleep(CRAWL_DELAY_SECONDS) # 各リクエスト後に待機
        soup = BeautifulSoup(response.text, "html.parser")

        for anchor in soup.find_all("a", href=True):
            href = anchor["href"]

            # JavaScriptリンクやアンカーリンク（ページ内ジャンプ）を除外
            if href.startswith("javascript:") or href.startswith("#"):
                continue

            # 絶対URLに変換
            absolute_url = urljoin(url, href)

            # 正規化
            normalized_absolute_url = normalize_url(absolute_url)

            # ドメインチェック
            base_hostname = urlparse(base_url).hostname
            link_hostname = urlparse(normalized_absolute_url).hostname

            # ホスト名が base_hostname またはそのサブドメインであるかチェック
            if link_hostname and (link_hostname == base_hostname or link_hostname.endswith('.' + base_hostname)):
                 links.add(normalized_absolute_url)

    except requests.exceptions.RequestException as e:
        print(f"Error fetching links from {url}: {e}")
    except Exception as e:
        print(f"Error processing links from {url}: {e}")
    return links

# メインループ: リンクを辿る (BFS)
while to_visit:
    # 巡回するURLを to_visit から一つ取り出す
    current_url = to_visit.pop()

    # 既に訪問済みならスキップ (これがないと無限ループや重複クロールが発生)
    if current_url in visited:
        continue

    # 訪問済みリストに加える
    visited.add(current_url)
    print(f"Visiting: {current_url} (Remaining to visit: {len(to_visit)}, Visited: {len(visited)})")

    # URLの<title>を辞書に格納
    page_title = get_title(current_url)
    url_title_dict[current_url] = page_title

    # ページ内のリンクを探索
    new_links = find_links(current_url)

    to_visit.update(new_links - visited)

# 結果を表示
print("\n--- Crawling Completed ---")
print("Total URLs found:", len(url_title_dict))
for url, title in url_title_dict.items():
    print(f"URL: {url}\nTitle: {title}\n---")

Visiting: https://www.musashino-u.ac.jp/ (Remaining to visit: 0, Visited: 1)
Visiting: https://www.musashino-u.ac.jp/research/interview/ (Remaining to visit: 98, Visited: 2)
Visiting: https://www.musashino-u.ac.jp/research/laboratory/institute_of_Law.html (Remaining to visit: 196, Visited: 3)


KeyboardInterrupt: 